### **Aplicando los Transformers para clasificación**



In [ ]:
# Librerías requeridas para este cuaderno 
# Las que ya vienen preinstaladas están comentadas.
# Si ejecutas el cuaderno en tu entorno local, descomenta las siguientes líneas:

# !pip install -q pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# - Para actualizar un paquete a la última versión disponible:
# !pip install pmdarima -U
# - Para fijar un paquete en una versión concreta:
# !pip install --upgrade pmdarima==2.0.2

# Nota: si tu entorno no soporta el comando "!pip install", deja estas líneas en comentarios.


In [ ]:
#!pip install dash-core-components==2.0.0 
#!pip install dash-table==5.0.0
#!pip install dash==2.9.3
#!pip install -Uqq dash-html-components==2.0.0
#!pip install -Uqq portalocker>=2.0.0
#!pip install -qq torchtext
#!pip install -qq torchdata
#!pip install -Uqq plotly

In [ ]:
# Puedes suprimir aqui los warnings generados por tu codigo
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn

from sklearn.manifold import TSNE

from torch.utils.data import DataLoader
import numpy as np
from torchtext.datasets import AG_NEWS
from IPython.display import Markdown as md
from tqdm import tqdm

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import pickle

from torch.nn.utils.rnn import pad_sequence

**Funciones auxiliares**

In [ ]:
def plot(COST,ACC):
    
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('epoch', color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis='y', color=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)  
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()  
    plt.show()

In [ ]:
def plot_embdings(my_embdings,name,vocab):
  
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  ax.scatter(my_embdings[:,0], my_embdings[:,1], my_embdings[:,2])

  for j, label in enumerate(name):
      i=vocab.get_stoi()[label]
      ax.text(my_embdings[j,0], my_embdings[j,1], my_embdings[j,2], label)

  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')

  plt.show()

In [ ]:
def plot_tras(words, modelo):
    tokens = tokenizer(words)

    d_model = 100

    x = torch.tensor(text_pipeline(words)).unsqueeze(0).to(device)

    x_ = modelo.emb(x) * math.sqrt(d_model)

    x = modelo.pos_encoder(x_)

    q_proj_weight = modelo.state_dict()['transformer_encoder.layers.0.self_attn.in_proj_weight'][0:embed_dim].t()
    k_proj_weight = modelo.state_dict()['transformer_encoder.layers.0.self_attn.in_proj_weight'][embed_dim:2*embed_dim].t()
    v_proj_weight = modelo.state_dict()['transformer_encoder.layers.0.self_attn.in_proj_weight'][2*embed_dim:3*embed_dim].t()

    Q = (x @ q_proj_weight).squeeze(0)
    K = (x @ k_proj_weight).squeeze(0)
    V = (x @ v_proj_weight).squeeze(0)

    scores = Q @ K.T

    row_labels = tokens
    col_labels = row_labels

    plt.figure(figsize=(10, 8))
    plt.imshow(scores.cpu().detach().numpy())
    plt.yticks(range(len(row_labels)), row_labels)
    plt.xticks(range(len(col_labels)), col_labels, rotation=90)
    plt.title("Atención producto-punto")
    plt.show()

    att = nn.Softmax(dim=1)(scores)
    plt.figure(figsize=(10, 8))
    plt.imshow(att.cpu().detach().numpy())
    plt.yticks(range(len(row_labels)), row_labels)
    plt.xticks(range(len(col_labels)), col_labels, rotation=90)
    plt.title("Atención producto-punto escalado")
    plt.show()

    head = nn.Softmax(dim=1)(scores) @ V

    tsne(x_, tokens, title="Embeddings")
    tsne(head, tokens, title="Cabeceras de atención")


def tsne(embeddings, tokens, title="Embeddings"):
    tsne = TSNE(n_components=2, random_state=0)
    tsne_result = tsne.fit_transform(embeddings.squeeze(0).cpu().detach().numpy())
    
    plt.scatter(tsne_result[:, 0], tsne_result[:, 1])

    plt.title(title)

    for j, label in enumerate(tokens):
        plt.text(tsne_result[j, 0], tsne_result[j, 1], label)

    plt.show()

In [ ]:
def save_list_to_file(lst, filename):
    with open(filename, 'wb') as file:
        pickle.dump(lst, file)

def load_list_from_file(filename):
    with open(filename, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

In [ ]:
dataset = [
    (1,"Introduction to NLP"),
    (2,"Basics of PyTorch"),
    (1,"NLP Techniques for Text Classification"),
    (3,"Named Entity Recognition with PyTorch"),
    (3,"Sentiment Analysis using PyTorch"),
    (3,"Machine Translation with PyTorch"),
    (1," NLP Named Entity,Sentiment Analysis,Machine Translation "),
    (1," Machine Translation with NLP "),
    (1," Named Entity vs Sentiment Analysis  NLP "),
    (3,"he painted the car red"),
    (1,"he painted the red car")
    ]

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for  _,text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
def text_pipeline(x):
    return vocab(tokenizer(x))

def label_pipeline(x):
    return int(x) - 1

#### **Cero padding**




In [ ]:
sequences = [torch.tensor([j for j in range(1,i)]) for i in range(2,10)]
sequences

In [ ]:
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
print(padded_sequences)

#### **Codificación posicional:**


In [ ]:
mi_tokens='he painted the car red he painted the red car'

mi_index=text_pipeline(mi_tokens)
mi_index

embedding_dim=3

vocab_size=len(vocab)
print(vocab_size)

embedding = nn.Embedding(vocab_size, embedding_dim)

In [ ]:
mi_embdings=embedding(torch.tensor(mi_index)).detach().numpy()
plot_embdings(mi_embdings,tokenizer(mi_tokens),vocab)

In [ ]:
position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
position

In [ ]:
d_model=3
pe = torch.zeros(vocab_size,d_model )

In [ ]:
pe=torch.cat((position, position, position), 1)
pe

In [ ]:
samples,dim=mi_embdings.shape
samples,dim

In [ ]:
pos_embding=mi_embdings+pe[0:samples,:].numpy()

In [ ]:
plot_embdings(pos_embding,tokenizer(mi_tokens),vocab)

In [ ]:
pos_embding[3]

In [ ]:
pos_embding[-1]

In [ ]:
pe=torch.cat((0.1*position, -0.1*position, 0*position), 1)

In [ ]:
plt.plot(pe[:, 0].numpy(), label="Dimension 1")
plt.plot(pe[:, 1].numpy(), label="Dimension 2")
plt.plot(pe[:, 2].numpy(), label="Dimension 3")

plt.xlabel("Número de secuencia")
plt.legend()
plt.show()

In [ ]:
pos_embding=mi_embdings+pe[0:samples,:].numpy()
plot_embdings(pos_embding,tokenizer(mi_tokens),vocab)

In [ ]:
pe=torch.cat((torch.sin(2*3.14*position/6), 0*position+1, 0*position+1), 1)
pos_embding=mi_embdings+pe[0:samples,:].numpy()
plot_embdings(pos_embding,tokenizer(mi_tokens),vocab)

In [ ]:
pe

In [ ]:
plt.plot(pe[:, 0].numpy(), label="Dimension 1", linestyle='-')
plt.plot(pe[:, 1].numpy(), label="Dimension 2", linestyle='--')
plt.plot(pe[:, 2].numpy(), label="Dimension 3", linestyle=':')

plt.ylim([-1, 1.1])

plt.xlabel("Número de secuencia")
plt.legend()
plt.show()

In [ ]:
pe=torch.cat((torch.cos(2*3.14*position/25), torch.sin(2*3.14*position/25),  torch.sin(2*3.14*position/5)), 1)
pos_embding=mi_embdings+pe[0:samples,:].numpy()
plot_embdings(pos_embding,tokenizer(mi_tokens),vocab)

In [ ]:
plt.plot(pe[:, 0].numpy(), label="Dimension 1")
plt.plot(pe[:, 1].numpy(), label="Dimension 2")
plt.plot(pe[:, 2].numpy(), label="Dimension 3")

In [ ]:
from torch import nn

class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [ ]:
mi_embdings=embedding(torch.tensor(mi_index))
mi_embdings

In [ ]:
mi_embdings.shape

In [ ]:
encoder_layer=nn.TransformerEncoderLayer(
            d_model=3,
            nhead=1,
            dim_feedforward=1,
            dropout=0,
        )

In [ ]:
out=encoder_layer(mi_embdings)
out

In [ ]:
out.mean(dim=1)

In [ ]:
params_dict = encoder_layer.state_dict()
for name, param in params_dict.items():
    print(name, param.shape)

In [ ]:
embed_dim=3
q_proj_weight = encoder_layer.state_dict()['self_attn.in_proj_weight'][0:embed_dim].t()
k_proj_weight = encoder_layer.state_dict()['self_attn.in_proj_weight'][embed_dim:2*embed_dim].t()
v_proj_weight = encoder_layer.state_dict()['self_attn.in_proj_weight'][2*embed_dim:3*embed_dim].t()

In [ ]:
Q=mi_embdings@q_proj_weight
K=mi_embdings@k_proj_weight
V=mi_embdings@v_proj_weight

In [ ]:
scores=Q@K.T/np. sqrt(embed_dim)
scores

In [ ]:
head=nn.Softmax(dim=1)(scores)@V
head

In [ ]:
transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=2)

Puedes mostrar la otra capa.


In [ ]:
params_dict = transformer_encoder.state_dict()
for name, param in params_dict.items():
    print(name, param.shape)

#### **Clasificación de texto**

In [ ]:
train_iter= AG_NEWS(split="train")

In [ ]:
y,text= next(iter(train_iter ))
print(y,text)

In [ ]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
ag_news_label[y]

In [ ]:
num_class = len(set([label for (label, text) in train_iter ]))
num_class

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
vocab(["age","hello"])

#### **Conjunto de datos**


In [ ]:
train_iter, test_iter = AG_NEWS()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)

split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

#### **Cargador de datos**


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        text_list.append(torch.tensor(text_pipeline(_text), dtype=torch.int64))


    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)


    return label_list.to(device), text_list.to(device)

In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

In [ ]:
label,seqence=next(iter(valid_dataloader ))

#### **Red neuronal**


In [ ]:
class Net(nn.Module):

    def __init__(
        
        self,
        vocab_size,
        num_class,
        embedding_dim=100,
        nhead=5,
        dim_feedforward=2048,
        num_layers=6,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1):

        super().__init__()

        self.emb = nn.Embedding(vocab_size,embedding_dim)

        self.pos_encoder = PositionalEncoding(
            d_model=embedding_dim,
            dropout=dropout,
            vocab_size=vocab_size,
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier = nn.Linear(embedding_dim, num_class)
        self.d_model = embedding_dim

    def forward(self, x):
        x = self.emb(x) * math.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.classifier(x)

        return x

In [ ]:
y,x=next(iter(train_dataloader))

In [ ]:
x

In [ ]:
emsize=64

In [ ]:
vocab_size=len(vocab)
vocab_size

In [ ]:
num_class

Creando el modelo:


In [ ]:
modelo = Net(vocab_size=vocab_size,num_class=4).to(device)
modelo

In [ ]:
predicted_label=modelo(x)

In [ ]:
predicted_label.shape

In [ ]:
x.shape

In [ ]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.unsqueeze(torch.tensor(text_pipeline(text)),0).to(device)

        output = modelo(text)
        return ag_news_label[output.argmax(1).item() + 1]

In [ ]:
predict("I like sports and stuff",text_pipeline )

In [ ]:
def evaluate(dataloader, model_eval):
    model_eval.eval()
    total_acc, total_count= 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model_eval(text.to(device))

            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [ ]:
evaluate(test_dataloader, modelo)

In [ ]:
# LR=0.1

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(modelo.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

Entrenando el modelo para 10 épocas.

>Omite este paso si no tienes GPU. Recupera y usa el modelo entrenado para 100 épocas y guardado en el siguiente paso.


In [ ]:
# EPOCHS = 10
# cum_loss_list=[]
# acc_epoch=[]
# acc_old=0

# for epoch in tqdm(range(1, EPOCHS + 1)):
#     modelo.train()
#     cum_loss=0
#     for idx, (label, text) in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         label, text=label.to(device), text.to(device)


#         predicted_label = modelo(text)
#         loss = criterion(predicted_label, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.1)
#         optimizer.step()
#         cum_loss+=loss.item()
#     print("Loss",cum_loss)

#     cum_loss_list.append(cum_loss)
#     accu_val = evaluate(valid_dataloader)
#     acc_epoch.append(accu_val)

#     if accu_val > acc_old:
#       acc_old= accu_val
#       torch.save(modelo.state_dict(), 'mi_modelo.pth')

# save_list_to_file(lst=cum_loss_list, filename="loss.pkl")
# save_list_to_file(lst=acc_epoch, filename="acc.pkl")

Tienes la capacidad de subir el modelo entrenado junto con datos completos sobre la pérdida acumulada y la precisión promedio en cada época.



### **Ejercicios**

0. **Integrar un modelo pre-entrenado (BERT) en el código existente**

   1. Instala la librería Transformers:

      ```bash
      pip install transformers
      ```
   2. En el cuaderno, sustituye la construcción de embeddings y el `TransformerEncoder` de la clase `Net` por un backbone de BERT:

      ```python
      from transformers import BertModel, BertTokenizer

      tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
      backbone  = BertModel.from_pretrained("bert-base-uncased")
      ```
   3. Modifica la clase `Net` para que reciba `backbone` en su constructor y, en `forward`, haga:

      ```python
      tokens = tokenizer(text,
                         padding="max_length",
                         truncation=True,
                         max_length=128,
                         return_tensors="pt")
      outputs = backbone(input_ids=tokens.input_ids.to(device),
                         attention_mask=tokens.attention_mask.to(device))
      pooled  = outputs.pooler_output           # [batch_size, hidden_size]
      logits  = self.classifier(self.dropout(pooled))
      ```
   4. Ajusta el `collate_batch` para devolver `text` en bruto y pasa el texto al modelo en lugar de tensores de índices.
   5. Entrena sobre AG\_NEWS durante **5 epocas** con `Adam(lr=2e-5)` y `criterion = CrossEntropyLoss()`.
   6. Al finalizar, imprime en pantalla:

      * **Shape de salida**: `(batch_size, 4)`

      * **Accuracy en test** (esperado): \~ 92 %

      * **Matriz de confusión** `4×4`

      * **Reporte de clasificación** (Precision/Recall/F1 por clase)

   > **Ejemplo de salida esperada**
   >
   > ```
   > Shape logits: torch.Size([32, 4])
   > Test accuracy: 0.91–0.93
   > Confusion matrix:
   > [[6121  120   50   30]
   >  [  80 5850  200   20]
   >  [  40  180 6000  100]
   >  [  35   15  125 6325]]
   > Classification report:
   >               precision    recall  f1-score   support
   >
   >            0       0.98      0.97      0.98      6321
   >            1       0.95      0.97      0.96      6050
   >            2       0.94      0.95      0.95      6320
   >            3       0.97      0.97      0.97      6500
   >
   >     accuracy                           0.92     25191
   >    macro avg       0.96      0.96      0.96     25191
   > weighted avg       0.96      0.92      0.96     25191
   > ```

1. **Habilitar el training loop y configurar optimizadores, pérdida y scheduler**

   1. Descomenta el bloque que define el bucle de entrenamiento.
   2. Configura al menos dos optimizadores distintos (por ejemplo, `SGD` y `Adam`) y compara sus resultados.
   3. Define `criterion = nn.CrossEntropyLoss()`.
   4. Añade un scheduler (por ejemplo, `StepLR` o `CosineAnnealingLR`) y muestra su estado en cada epoch.
   5. Ejecuta el entrenamiento durante **10 epocas**, registrando pérdida y precisión de validación en listas.
   6. Usa `plot(COST, ACC)` para graficar las curvas de entrenamiento y validación.

2. **Guardado y recarga de checkpoints**

   1. Al final de cada epoca, comprueba si la precisión en validación ha mejorado.
   2. Si ha mejorado, guarda un checkpoint:

      ```python
      torch.save({
          'epoch': epoch,
          'model_state': modelo.state_dict(),
          'optimizer_state': optimizer.state_dict(),
          'best_acc': best_acc,
      }, 'checkpoint_best.pth')
      ```
   3. Implementa `load_checkpoint(path, modelo, optimizer=None)` para recargar modelo y optimizador.
   4. Detén el entrenamiento tras 5 epocas, recarga el checkpoint y continúa 5 epocas más, verificando que las métricas sean consistentes.


3. **Métricas avanzadas y logging en TensorBoard**

   1. Durante la evaluación, además de `accuracy`, calcula:

      * Matriz de confusión (`sklearn.metrics.confusion_matrix`).
      * Precision, Recall y F1-score por clase (`sklearn.metrics.classification_report`).
   2. Inicializa un `SummaryWriter` y registra en cada epoch:

      * Scalars: `loss_train`, `loss_val`, `acc_train`, `acc_val`.
      * Matriz de confusión como imagen.
      * Reporte de clasificación como texto o tabla.
   3. Ejecuta `tensorboard --logdir=runs` y verifica los gráficos y métricas.


4. **Más robustez en DataLoader y máscaras de padding**

   1. Modifica `collate_batch` para devolver también `src_key_padding_mask` de forma `(batch_size, seq_len)`.
   2. Pasa esta máscara al `TransformerEncoder`:

      ```python
      out = encoder(src_emb, src_key_padding_mask=padding_mask)
      ```
   3. Asegúrate de que los tokens de padding no participen en la atención.
   4. Compara la estabilidad de la pérdida con y sin máscara para validar la corrección.


5. **Parametrización de hiperparámetros**

   1. Añade `argparse` para recibir:

      * `--num-layers`, `--num-heads`, `--dim-feedforward`, `--dropout`
      * `--batch-size`, `--lr`, `--epochs`
   2. Alternativamente, carga un archivo JSON o YAML con los mismos parámetros.
   3. Inicializa modelo, optimizador y dataLoaders a partir de estos valores.
   4. Ejecuta el script con distintos parámetros y observa el impacto en el rendimiento.


6. **Entrenamiento distribuido (DistributedDataParallel)**

   1. Inicializa el entorno distribuido con `torch.distributed.init_process_group`.
   2. Envuelve el modelo en `torch.nn.parallel.DistributedDataParallel`.
   3. Usa `DistributedSampler` en los DataLoaders.
   4. Entrena en 2 o más procesos y verifica que el rendimiento escala con el número de GPUs/processes.


7. **Empaquetar el modelo como API de inferencia**

   1. Elige **FastAPI** o **Flask**.
   2. Carga el modelo desde el checkpoint.
   3. Define un endpoint `/predict/` que acepte JSON `{ "text": "..." }`.
   4. Aplica `text_pipeline`, construye el tensor, ejecuta `modelo.eval()` y devuelve `{ "label": int, "score": float }`.
   5. Prueba la API con `curl` o un script en `requests`.


8. **Reproducibilidad**

   1. Al inicio del script, fija semillas:

      ```python
      seed = 42
      torch.manual_seed(seed)
      np.random.seed(seed)
      random.seed(seed)
      if torch.cuda.is_available():
          torch.cuda.manual_seed_all(seed)
      ```
   2. Ejecuta dos entrenamientos idénticos y compara métricas para asegurar consistencia.

9. **Documentación y pruebas unitarias con pytest**

   1. Añade docstrings a:

      * `text_pipeline()`, `label_pipeline()`, `PositionalEncoding`, `Net.forward()`.
   2. Crea tests que verifiquen:

      * `text_pipeline("foo bar")` devuelve lista de enteros.
      * `PositionalEncoding(d_model=4, max_len=10)` genera tensor `(1, 10, 4)`.
      * `Net.forward()` con batch sintético devuelve logits `(batch_size, num_classes)`.
   3. Asegura cobertura mínima del 80 %.


10. **Uso de un modelo pre-entrenado de Hugging Face para clasificación**

    1. Carga `bert-base-uncased` con `BertModel.from_pretrained(...)`.
    2. Reemplaza embedding y encoder por BERT, congelando o descongelando últimas capas.
    3. Entrena sobre AG\_NEWS y compara precisión y tiempos con el Transformer desde cero.


11. **Full fine-tuning vs. fine-tuning parcial**

    1. Experimento 1: ajusta todos los parámetros de BERT.
    2. Experimento 2: ajusta solo la capa de clasificación.
    3. Compara rendimiento y coste computacional en ambos casos.


12. **Implementación de LoRA en un Transformer de clasificación**

    1. Aplica low-rank adaptation a las proyecciones Q y V de `TransformerEncoderLayer`.
    2. Añade matrices de bajo rango aprendibles.
    3. Evalúa impacto en precisión y número de parámetros.


13. **Adaptación de QLoRA para texto largo**

    1. Cuantiza tu modelo pre-entrenado a 4 bits usando QLoRA.
    2. Entrena con batch pequeño y observa cambios en memoria y precisión.

14. **Visualización de mapas de atención**

    1. Extiende `plot_tras` para múltiples cabeceras y capas.
    2. Para muestras de test, dibuja atención y describe patrones relevantes.

15. **Evaluación con RLHF simplificado**

    1. Genera predicciones en validación.
    2. Simula una función de recompensa que penalice errores en clases críticas.
    3. Ajusta el modelo con policy gradient (por ejemplo, REINFORCE).
    4. Compara con el entrenamiento supervisado estándar.


16. **Experimentación con objetivos de preentrenamiento**

    1. Preentrena un encoder pequeño con MLM y CLM sobre un corpus reducido.
    2. Transfiérelo a clasificación y compara con un modelo entrenado desde cero.

17. **Benchmark de distilación y cuantización**

    1. Aplica knowledge distillation: entrena un modelo student para imitar un teacher grande.
    2. Posteriormente aplica post-training quantization al student.
    3. Mide precisión vs. tamaño del modelo.

18. **Exploración de emergent abilities al aumentar capas**

    1. Crea instancias de `Net` con 2, 4, 8 y 12 capas.
    2. Entrena brevemente cada una.
    3. Grafica precisión vs. número de parámetros y observa posibles saltos de rendimiento inesperados.


In [ ]:
### Tus respuestas